In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
#import qiskit

### BB84 Recap
- Two random bit strings for A: `a`, `b`
- `bᵢ` decides which basis `aᵢ` is encoded in → `ψ`
- A sends `ψ` over a public quantum channel
- B randomly generates `b'` to decode received qubits
- B gets the message `a'` and publicly announces he received it
- A then publicly releases `b` and B releases `b'`
- They compare `b'` and `b`:
  - Bits in `a` and `a'` are discarded if `b'` and `b` don't match
- From the remaining bits, `k/2` are chosen
- A and B publicly announce these bits to check:
  - if sufficiently many match, you've successfully generated a key


In [3]:
#generate Alice bits and basis choices
def Alice_setup(n):
    bits = ""
    bases = ""

    for i in range(n):
        bits += str(random.randint(0,1))
        bases += str(random.randint(0,1))
    return bits,bases

#generate random basis choices for Bob
def Bob_setup(n):
    bobs_bases = ""
    for i in range(n):
        bobs_bases += str(random.randint(0,1))

    return bobs_bases


In [4]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator

def encode(a,b):
    
    # the quantum state is prepared according, encoding a in base b

    qc = QuantumCircuit(len(a))

    for i in range(len(a)):

        #computational basis
        if b[i] == "0":
            
            #|0>
            if a[i] == "0":
                qc.id(i)
            # |1> 
            else:
                qc.x(i)
                
        #+/- basis
        else:
            #|+>
            if a[i] == "0":
                qc.h(i)
            #|->
            else:
                qc.x(i)
                qc.h(i)

    # execute circuit then return the state 
    sim = AerSimulator(method='statevector')
    qc.save_statevector()
    state = sim.run(qc).result().get_statevector(qc)

    return state



In [5]:
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError


def noisy_fast_channel(input_state, p_depol=0.05, p_readout=0.02):

    # this function models a noisy quantum channel in a computationally efficient way
    # at least more efficient than with kraus operators
    
    n = input_state.num_qubits

    # build noise model with readout and depolarizing noise
    noise_model = NoiseModel()
    depol_error = depolarizing_error(p_depol, 1)
    readout_error = ReadoutError([[1 - p_readout, p_readout],
                                  [p_readout, 1 - p_readout]])
    noise_model.add_all_qubit_quantum_error(depol_error, ["id", "x", "h"])
    noise_model.add_all_qubit_readout_error(readout_error)

    # create circuit with initial state
    qc = QuantumCircuit(n, n)
    qc.initialize(input_state, range(n))
    qc.measure(range(n), range(n))

    # simulate with noise
    simulator = AerSimulator(noise_model=noise_model)
    result = simulator.run(qc, shots=1, memory=True).result()
    return result.get_memory(qc)[0]


In [6]:
from qiskit import ClassicalRegister,transpile
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

def decode(received_state, bases):

    # Bob decodes the state he receives with his basis choices

    # initialize a quantum circuit with received info
    qcirc = QuantumCircuit(len(bases))
    qcirc.prepare_state(received_state,range(len(bases)))

    # add classical bits to store measurement
    qcirc.add_register(ClassicalRegister(len(bases), "c"))

    # decide in which basis to measure
    for i in range(len(bases)):
        #computational basis
        if bases[i] == "0":
            qcirc.measure(i, i)

        # +\- basis
        else:
            qcirc.h(i)
            qcirc.measure(i,i)

    #execute circuit

    #transpile for simulator
    simulator = AerSimulator()
    circ = transpile(qcirc, simulator)

    #run once and return the result
    results = simulator.run(circ, shots=1, memory=True).result()
    memory = results.get_memory(circ)
    return memory[0][::-1] # because qiskit reads them the other way round



In [7]:
#B announces that he got the message and A rpublicly eleases her bases-string 

def post_processing1(a_bits, a_bases, b_bits, b_bases):
    
    # sifting. this returns the key, so all bits where Alice and Bobs bases matched

    a_bits_use = ""
    b_bits_use = ""
    #keep only the bits where bases matched
    for i in range(len(a_bits)):
        if a_bases[i] == b_bases[i]:
            a_bits_use += a_bits[i]
            b_bits_use += b_bits[i]
        
    return a_bits_use, b_bits_use


In [8]:
# and now in the grand finale, all together:
def bb84_theory(n):

    # full simulation of the bb84 protocol without noise or evesdropper

    alice_bits, alice_bases = Alice_setup(n)
    sent_state = encode(alice_bits, alice_bases)
    
    bobs_bases = Bob_setup(n)
    bobs_bits = decode(sent_state, bobs_bases)
    alice_key, bobs_key = post_processing1(alice_bits, alice_bases, bobs_bits, bobs_bases)
    
    #key comparision
    print("Alice key: "+alice_key)
    print("Bobs key:  "+bobs_key)
    return



In [ ]:
bb84_theory(20)

In [ ]:

def noisy_bb84(n):

    # this simulates the bb84 protocol with a noisy channel
    
    alice_bits, alice_bases = Alice_setup(n)
    sent_state = encode(alice_bits, alice_bases)

    #channel
    received_state = noisy_fast_channel(sent_state)
    
    #bob
    bobs_bases = Bob_setup(n)
    bobs_bits = decode(received_state, bobs_bases)
    alice_key, bobs_key = post_processing1(alice_bits, alice_bases, bobs_bits, bobs_bases)

    #output
    print("Alice key: "+alice_key)
    print("Bobs key:  "+bobs_key)
    return

In [ ]:
noisy_bb84(20)

Alice key: 0110001001100
Bobs key:  0101010011110


In [ ]:
def evesdropper(intercepted_state, num_picked):

    # model the behaviour of the evesdropper

    # safety check the variables
    num_intercepted_qubits = intercepted_state.num_qubits
    if num_picked > num_intercepted_qubits:
        raise ValueError("Intercepted state doesn't contain enough qubits")

    # initialize the circuit from intercepted state
    qc = QuantumCircuit(num_intercepted_qubits)
    qc.set_statevector(intercepted_state)
    qc.add_register(ClassicalRegister(num_picked, "c"))

    # determine equidistant qubit positions for measurement
    picked_indices = list(np.linspace(0, num_intercepted_qubits - 1, num_picked, dtype=int))
    

    eve_bases = ""
    # randomly generate basis choices:
    for i in range(num_picked):
        eve_bases += str(random.randint(0,1))
    
    # Measure chosen qubits into classical bits
    for i, qubit_idx in enumerate(picked_indices):
        if eve_bases[i] == "0":
            qc.measure(qubit_idx, i)
        else:
            qc.h(qubit_idx)
            qc.measure(qubit_idx,i)
    
    qc.save_statevector(label="after_measure")
    
    # Simulate
    sim = AerSimulator()
    result = sim.run(qc, shots=1, memory=True).result()
    forwarded_state = result.data(0)["after_measure"]
    eve_bits = result.get_memory()[0][::-1]

    return forwarded_state, eve_bits, eve_bases

In [ ]:
def post_processing_eve(eve_bits, eve_bases, alice_bases, n,k):

    #visually compare what the evesdropper receives compared to alice and bobs bits 
    
    picked_indices = list(np.linspace(0, n - 1, k, dtype=int))
    
    #visually display which qubits have been measured and which of those are correct (correct choice of basis)
    visual_eve_bits = ""
    idx=0
    #iterate over all sent qubits
    for i in range(n):
        #check if qubit was intercepted
        if i in picked_indices:
            #check if it was measured in the correct basis
            if alice_bases[i] != eve_bases[idx]:
                visual_eve_bits += "."
            else:
                visual_eve_bits += eve_bits[idx]
            idx +=1

        else:
            visual_eve_bits += " "

        
    return visual_eve_bits

        
#post_processing_eve("11000", "11010", "1010011110111000100", 20, 5)

In [ ]:
def post_processing_eve_key(eve_bits_vis, alice_bases, bob_bases):
    
    # generate a visual representation of which bits of the key the evesdropper has successfully intercepted

    eve_key = ""
    i=0
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i]:
            eve_key += eve_bits_vis[i]
    return eve_key


In [ ]:
def post_processing2(alice_key, bob_key):
    
    # return the error rate of how many bits btw alice and bob

    mismatched_bits = 0
    for i in range(len(alice_key)//2):
        if int(alice_key[i]) != int(bob_key[i]):
            mismatched_bits +=1
    
    return mismatched_bits/(len(alice_key)//2)

In [ ]:
def bb84_with_evesdropper(n,k):

    # bb84 protocol with evesdropper but without noisy channel

    #alice prepares and sends out state
    alice_bits, alice_bases = Alice_setup(n)
    sent_state = encode(alice_bits, alice_bases)

    #state is intercepted and forwarded to bob
    forwarded_state, eve_bits, eve_bases = evesdropper(sent_state, k)
    
    #bob measures what he receives
    bobs_bases = Bob_setup(n)
    bobs_bits = decode(forwarded_state, bobs_bases)

    #for visual comparison, the bits the evesdropper has intercepted
    visual_eve_bits = post_processing_eve(eve_bits, eve_bases, alice_bases, n,k)
    
    print("before sifting: ")
    print("Alice bits: "+alice_bits)
    print("Bobs bits:  "+bobs_bits)
    print("Eves bits:  "+visual_eve_bits+"\n")

    #sifting process, for evesdropper as well
    alice_key, bobs_key = post_processing1(alice_bits, alice_bases, bobs_bits, bobs_bases)
    eve_key = post_processing_eve_key(visual_eve_bits, alice_bases, bobs_bases)

    print("after sifiting: ")
    print("Alice key: "+alice_key)
    print("Bob key:   "+bobs_key)
    print("Eve key:   "+eve_key+"\n")

    #sacrifical lamb 
    #alice and bob sacrifice half of the key to check if their bits match
    error_rate = post_processing2(alice_key, bobs_key)
    print("Error rate: "+ str(error_rate))

    return
    

In [ ]:
bb84_with_evesdropper(20,5)

before sifting: 
Alice bits: 11010010111101110011
Bobs bits:  01000011011001100001
Eves bits:  .   .    .    .    1

after sifiting: 
Alice key: 10111010
Bob key:   10111010
Eve key:    . .    

Error rate: 0.0
